In [1]:
#Imports y carga del dataset
import pandas as pd
import math
import io

pd.set_option("display.max_rows", None)

with open("movies_dataset/movies_metadata.csv", "r", encoding='utf-8') as f:
    df_raw = pd.read_csv(f, encoding='utf-8')

c:\users\raulb\documents\universidad\4º_curso\primercuatrimestre\mineria_de_datos\práctica\mineria-datos\venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#Excluir columnas que no vamos a usar
exclude = ["homepage", "imdb_id", "original_title", "poster_path", "status", "video", "id"] 
df = df_raw.loc[:, df_raw.columns.difference(exclude)]

In [3]:
#Comprobación de valores nulos
print(pd.isnull(df).any())
print()
print("Filas totales:", len(df))
print("Filas con algún valor nulo:", len([x for x in df.isnull().any(axis=1) if x]))

adult                    False
belongs_to_collection     True
budget                   False
genres                   False
original_language         True
overview                  True
popularity                True
production_companies      True
production_countries      True
release_date              True
revenue                   True
runtime                   True
spoken_languages          True
tagline                   True
title                     True
vote_average              True
vote_count                True
dtype: bool

Filas totales: 45466
Filas con algún valor nulo: 42894


In [5]:
'''ADULT'''

#Aqui observamos que hay valores no booleanos que debemos eliminar, y además observamos que
#las películas con valor True son muy escasas en comparación con las que tienen valor False,
#por lo que vamos a eliminar las filas anómalas y quitar la columna "adult"

print(df["adult"].value_counts())
print()

drop_list = []
for i in range(len(df)):
    if df["adult"][i] != "False":
        drop_list.append(i)
df.drop(index=drop_list, inplace=True)
print(df["adult"].value_counts())
print()

df.drop(columns=["adult"], inplace=True)
print(list(df))

False                                                                                                                             45454
True                                                                                                                                  9
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
 - Written by Ørnås                                                                                                                   1
Name: adult, dtype: int64

False    45454
Name: adult, dtype: int64

['belongs_to_collection', 'budget', 'genres', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline', 'title', 'vote_average', 'vote_count']


In [6]:
# Definition of functions for processing natural language columns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

replaces = [("i'm", "i am"),
            ("you're", "you are"),
            ("we're", "we are"),
            ("they're", "they are"),
            
            ("that's", "that is"),
            ("there's", "there is"),
            ("who's", "who is"), 
            ("what's", "what is"),
            
            ("isn't", "is not"),
            ("aren't", "are not"), 
            ("can't", "cannot"),
            ("haven't", "have not"),
            ("won't", "will not"),
            ("doesn't", "does not"),
            ("don't", "do not"),
            ("didn't", "did not"),
            ("couldn't", "could not"),
            ("shouldn't", "should not"),
            ("wasn't", "was not"),
            ("weren't", "were not"),
            ("hasn't", "has not"),
            ("hadn't", "had not"),
            
            ("we'd", "we would"),
            ("he'd", "he would"),
            ("you'd", "you would"),
            ("they'd", "they would"), 
            ("who'd", "who would"),
            ("wouldn't", "would not"),
            
            ("i'll", "i will"),
            ("you'll", "you will"),
            ("he'll", "he will"),
            ("she'll", "she will"),
            ("it'll", "it will"),
            ("we'll", "we will"),
            ("they'll", "they will"),
            
            ("i've", "i have"),
            ("you've", "you have"),
            ("we've", "we have"),
            ("they've", "they've"),
            ("&", " and ")
           ]

def sub_contractions(text):
    new_text = text
    for r in replaces:
        new_text = re.sub(f"{r[0]}", f"{r[1]}", new_text)
    return new_text

lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    if type(text) is float:  #Comprobando valores nulos, expresados como un float NaN
        ret_text = []
    else:
        new_text = text.lower()
        new_text = sub_contractions(new_text)
        new_text = re.sub("[^A-Za-zÀ-ÖØ-öø-ÿ0-9 ]", "", new_text)
        
        tok_text = word_tokenize(new_text)
        stop_words = list(stopwords.words("english"))
        tok_text = [word for word in tok_text if word not in stop_words]
        tok_text = [lemmatizer.lemmatize(word) for word in tok_text]
        ret_text = tok_text
        
        """ Processing feedback
        joint_text = " ".join(tok_text)
        print(new_text)
        print(joint_text)
        print() #"""
        
    return ret_text

In [5]:
# title, tagline, overview y belongs_to_collection

In [ ]:
'''TITLE'''

df["preprocessed_title"] = df["title"].apply(preprocess_text)
df["preprocessed_title"].head()

In [ ]:
'''TAGLINE'''

df["preprocessed_tagline"] = df["tagline"].apply(preprocess_text)
df["preprocessed_tagline"].head()

In [9]:
'''OVERVIEW'''

df["preprocessed_overview"] = df["overview"].apply(preprocess_text)
df["preprocessed_overview"].head()

0        [led, woody, andys, toy, live, happily, room, ...
1        [sibling, judy, peter, discover, enchanted, bo...
2        [family, wedding, reignites, ancient, feud, ne...
3        [cheated, mistreated, stepped, woman, holding,...
4        [george, bank, recovered, daughter, wedding, r...
5        [obsessive, master, thief, neil, mccauley, lea...
6        [ugly, duckling, undergone, remarkable, change...
7        [mischievous, young, boy, tom, sawyer, witness...
8        [international, action, superstar, jean, claud...
9        [james, bond, must, unmask, mysterious, head, ...
10       [widowed, u, president, andrew, shepherd, one,...
11       [lawyer, show, vampire, doorstep, fall, prey, ...
12       [outcast, halfwolf, risk, life, prevent, deadl...
13       [allstar, cast, power, epic, look, american, p...
14       [morgan, adam, slave, william, shaw, quest, re...
15       [life, gambling, paradise, la, vega, dark, maf...
16       [rich, mr, dashwood, dy, leaving, second, wife.

In [10]:
#Fecha dejar a nulo, no eliminar
#Pais / idioma variables categoricas, agrupar en regiones y one hot encoding
#Objetivo es convertir en variables numericas, añadiendo columnas por one hot encoding

In [1]:
'''VOTE_COUNT'''

# Eliminamos las filas con valores nulos
df.drop(df[df.vote_count.isna()].index, inplace= True)
# Si el numero de votos es menor a 1 no es útil, puesto que una pelicula sin votos no sirve
# para deducir en función de la media de votos de una pelicula
df = df[df.vote_count >= 1]

'''VOTE_AVERAGE'''

# Eliminamos las filas con valores nulos
df.drop(df[df.vote_average.isna()].index, inplace= True)
# De las filas anteriores, es decir, que tengan votos significativos, considerar todos los 
# votos (Hay filas que tienen una media de 0 pero ningun voto, por lo que no son válidas)

'''RELEASE_DATE'''

# Eliminamos las filas con valores nulos
df.drop(df[df.release_date.isna()].index, inplace= True)
# La columna de las fechas estaba en string (probablemente debido a los valores erroneos que había al principio)
# asi que la convertimos en una columna de tipo datetime
df['release_date'] = pd.to_datetime(df['release_date'])
# Podemos dejarlos como están o bien agrupar los años para extraer conclusiones
df_years = df['release_date'].groupby(df.release_date.dt.year)
#print(df_years.first())

'''RUNTIME'''

# Eliminamos las filas con valores nulos
df.drop(df[df.runtime.isna()].index, inplace= True)
# Borrar los que tienen valor 0 porque no tiene sentido que una pelicula tenga 0 minutos 
df = df[df.runtime > 0]

print(pd.isnull(df).any())
print()
print(len(df))
print()

NameError: name 'df' is not defined

In [ ]:
#Para elegir los idiomas validos vamos a ver los 6 mas usados y juntar los otros en la categoria resto

#print(pd.isnull(df).any())
print(df["original_language"].value_counts(dropna=False))

#Cambiar filas con idiomas inválidos
idiomas_validos = {'en', 'fr', 'it', 'ja', 'de', 'es'}

for nodo in df:
   if original_language not in idiomas_validos 
      original_language = other

In [ ]:
#Para elegir los idiomas validos vamos a ver los 6 mas usados y juntar los otros en la categoria resto

#print(pd.isnull(df).any())
print(df["spoken_language"].value_counts(dropna=False))


#Cambiar filas con idiomas inválidos
idiomas2_validos = {'en', 'fr', 'it', 'ja', 'de', 'es'}


for nodo in df:
   if original_language not in idiomas2_validos 
      original_language = other

In [ ]:
#Para elegir los generos validos vamos a ver los 5 mas usados y juntar los otros en la categoria resto

#print(pd.isnull(df).any())
print(df["genres"].value_counts(dropna=False))

#Camabiar filas con generos inválidos
generos_validos = {'1', '2', '3', '4', '5'}

for nodo in df:
   if genres not in generos_validos 
      genres = other



In [ ]:
#Tenemos que elegir entre tocar o no el presupuesto, si lo dejamos tal cual o en 4 categorias que habia dudas  (que sean super produccion, gran produccion , produccion normal y produccion bajo presupuesto por ejemplo)